In [1]:
import os
os.chdir(r"C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification")
print("Current directory:", os.getcwd())

Current directory: C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification


In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [4]:
from cnnProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnnProject.utils.common import readYamlFile,createDirectories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=readYamlFile(config_file_path)
        self.params=readYamlFile(PARAMS_FILE_PATH)
        createDirectories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        createDirectories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import urllib.request as request 
from cnnProject import logger
from cnnProject.utils.common import getSize
import zipfile

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
             filename,headers=request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_file)
             logger.info(f"{filename} downloaded Sucessfully with the  {headers} info")

        else:
            logger.info(f"file already exists of size {getSize(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(name=unzip_path,exist_ok=True)
        with zipfile.ZipFile(file=self.config.local_data_file,mode='r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-28 18:09:30,951 : INFO : common : the yaml file form the path config\config.yaml loaded Sucessfully]
[2025-06-28 18:09:30,957 : INFO : common : the yaml file form the path params.yaml loaded Sucessfully]
[2025-06-28 18:09:30,959 : INFO : common : Directories created Sucessfully at artifacts]
[2025-06-28 18:09:30,960 : INFO : common : Directories created Sucessfully at artifacts/data_ingestion]
[2025-06-28 18:09:35,344 : INFO : 1491931832 : artifacts/data_ingestion/data.zip downloaded Sucessfully with the  Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4250:30DE2A:8EA1B:1A0AA3:685FE283
Accept-Ranges: bytes
Date: Sat, 28 Jun 202